## 418. Sentence Screen Fitting
- Description:
  <blockquote>
  Given a `rows x cols` screen and a `sentence` represented as a list of strings, return _the number of times the given sentence can be fitted on the screen_.

  The order of words in the sentence must remain unchanged, and a word cannot be split into two lines. A single space must separate two consecutive words in a line.

  **Example 1:**

  ```
  Input: sentence = ["hello","world"], rows = 2, cols = 8
  Output: 1
  Explanation:
  hello---
  world---
  The character '-' signifies an empty space on the screen.

  ```

  **Example 2:**

  ```
  Input: sentence = ["a", "bcd", "e"], rows = 3, cols = 6
  Output: 2
  Explanation:
  a-bcd- 
  e-a---
  bcd-e-
  The character '-' signifies an empty space on the screen.

  ```

  **Example 3:**

  ```
  Input: sentence = ["i","had","apple","pie"], rows = 4, cols = 5
  Output: 1
  Explanation:
  i-had
  apple
  pie-i
  had--
  The character '-' signifies an empty space on the screen.

  ```

  **Constraints:**

  -   `1 <= sentence.length <= 100`
  -   `1 <= sentence[i].length <= 10`
  -   `sentence[i]` consists of lowercase English letters.
  -   `1 <= rows, cols <= 2 * 10<sup>4</sup>`
  </blockquote>

- URL: [Problem_URL](https://leetcode.com/problems/sentence-screen-fitting/description/)

- Topics: Problem_topic

- Difficulty: Medium

- Resources: example_resource_URL

### Solution 1, TLE, Brute Force Simulation


The reason we do not do + 1 in the while loop check:
A word only needs a space between it and the next word. It does not need a space if it is the very last word to fit in the row. 
By checking only the word length first, we allow words to fit exactly into the remaining space.

The reason we don't include the + 1 in the while check is to handle the exact fit case.

Consider this example:

    cols = 5
    word = "apple" (length 5)

1. If we check curr_col + len(word) <= cols:

    0 + 5 <= 5 is True.
    The word "apple" fits perfectly in the row.
    Then we set curr_col = 5 + 1 = 6. The next word will check 6 + len(next_word) <= 5, which correctly fails, moving the next word to a new row.

2. If we check curr_col + len(word) + 1 <= cols:

    0 + 5 + 1 <= 5 is False.
    The simulation would think "apple" (length 5) cannot fit in a 5-column row because it is looking for enough space for the word plus a trailing space.


- Time Complexity: O(rows * cols​ / L) ~ O(rows * cols)
  - Rows: The outer loop runs exactly rows times.
  - Inner Loop: For each row, the while loop runs for as many words as can fit in cols. If the average length of a word is L, you fit approximately Lcols​ words per row.
  - Worst Case: If every word is length 1 and cols is large, the inner loop runs O(cols) times. Therefore, the total time complexity is O(rows * cols).

- Space Complexity: O(1)
  - This approach only uses a few integer variables (count, word_idx, curr_col) regardless of the input size.

In [ ]:
class Solution:
    def wordsTyping(self, sentence: List[str], rows: int, cols: int) -> int:
        n = len(sentence)
        count = 0
        word_idx = 0
        
        for _ in range(rows):
            curr_col = 0
            # Try to fit as many words as possible in the current row
            while curr_col + len(sentence[word_idx]) <= cols:
                # Add word length + 1 for the space
                curr_col += len(sentence[word_idx]) + 1
                word_idx += 1
                
                # If we finish the sentence, increment count and reset index
                if word_idx == n:
                    count += 1
                    word_idx = 0
                    
        return count

### Solution 2, DP / Memoization (Row-by-Row Precomputation)

- Core Idea: If a row starts with a specific word index from your sentence, the outcome (how many words fit in that row and which word index starts the next row) will always be the same.

- Goal: For each word index i in the sentence, precompute how many times the sentence wraps around if you start a row with sentence[i].

- Advantage: Once precomputed, you can jump through the rows in O(rows) time.


For each word index i in the sentence, we precompute:
1. How many times the full sentence is completed in that single row.
2. The index of the word that will start the next row.



##### Why this is better than simple simulation:

In a **standard simulation**, you process the "words per row" calculation for _every_ row. This results in O(rows×Lcols).  
If `rows = 1,000,000` and `n = 100`, the DP approach only does the "heavy lifting" 100 times, whereas the simulation does it 1,000,000 times!


### Complexity Analysis

- Time Complexity: O(N * cols​/L + rows)
  - Where n is the number of words. We only simulate the row logic for each word index once.
  - Precomputation / Filling the Memo: O(N * cols​/L)
    - Outer part: There are n unique word indices that can start a row. We calculate the result for each index exactly once.
    - Inner part (The while loop): For each starting index, we simulate filling one row of width cols.
    - How many words can fit in a row? If the average length of a word is L, you can fit approximately cols​/L words.
    - In the worst case (where every word is 1 character long), you fit cols​/2 words (counting the spaces).
    - Total for this phase: (Number of words n) × (Words per row Lcols​).
  - The Main Loop (Jumping): O(rows)
    - Once the memo is populated (or as we fill it lazily), the main loop runs for the number of rows specified.
    - In each iteration of the rows loop, we do a constant time O(1) lookup: count, next_idx = memo[curr_word_idx].
    - Even if rows is 10**9, this is just a simple loop that increments a counter and updates an index.

- Space Complexity: O(N)
  - To store the memo dictionary.

In [ ]:
class Solution:
    def wordsTyping(self, sentence: List[str], rows: int, cols: int) -> int:
        n = len(sentence)
        memo = {} # word_index -> (count_of_sentences_completed_in_row, next_word_index)
        
        total_sentences = 0
        curr_word_idx = 0
        
        # Since words cannot be split, If a single word in the sentence is longer than cols
        # it can never be placed on the screen. Because the sentence must be typed in order, 
        # if you can't fit that one word, you can't finish the sentence even once.
        for word in sentence:
            if len(word) > cols:
                return 0
        
        for _ in range(rows):
            # If we haven't seen this starting word before, calculate the row result
            if curr_word_idx not in memo:
                row_count = 0
                curr_col = 0
                temp_idx = curr_word_idx
                
                while curr_col + len(sentence[temp_idx]) <= cols:
                    curr_col += len(sentence[temp_idx]) + 1
                    temp_idx += 1
                    if temp_idx == n:
                        row_count += 1
                        temp_idx = 0
                
                memo[curr_word_idx] = (row_count, temp_idx)
            
            # Use the memoized result to jump to the next row
            count, next_idx = memo[curr_word_idx]
            total_sentences += count
            curr_word_idx = next_idx
            
        return total_sentences

### DP with Cycle Detection
Cycle detection is the ultimate optimization for problems with repeating states. Here is how it works for this problem:

### 1. The Finite States
In the DP approach, each row starts with a specific word index from the sentence. Since there are only `n` words in the sentence, there are only `n` possible "starting states" for any row.

### 2. The Pigeonhole Principle
If you have $n$ words and you process more than $n$ rows, you **must** eventually start a row with a word index you have seen before. 
*   Example: Row 2 starts with word index `3`. Later, Row 10 also starts with word index `3`. 
*   This means Rows 2 through 9 form a **cycle**.

### 3. Jumping the Rows
Once a cycle is detected, you know:
*   **Cycle Length:** How many rows the cycle lasts (e.g., $10 - 2 = 8$ rows).
*   **Cycle Gain:** How many full sentences were completed during those 8 rows.

Instead of simulating the next million rows one by one, you can use math:
1.  **Remaining Rows:** `rem = total_rows - current_row`
2.  **Number of Cycles:** `num_cycles = rem // cycle_length`
3.  **Fast Forward:** `total_sentences += num_cycles * cycle_gain`
4.  **Finish Up:** Simulate the few remaining rows (`rem % cycle_length`) manually.

### Why this is powerful:
With cycle detection, the `rows` loop effectively stops as soon as a repeat is found (at most after `n + 1` iterations). This turns the time complexity from $O(\text{rows})$ into **$O(n \times \frac{\text{cols}}{L})$**. 

If `rows` is **1 quadrillion ($10^{15}$)**, the String Transformation approach would take years to run, but the Cycle Detection approach would finish in milliseconds.

Does the idea of using the "repeat" to multiply the result make sense, or would you like to see a small numerical example?

In [ ]:
class Solution:
    def wordsTyping(self, sentence: List[str], rows: int, cols: int) -> int:
        n = len(sentence)
        memo = {}  # word_idx -> (sentences_in_row, next_word_idx)
        seen = {}  # word_idx -> (row_index, total_sentences_at_start)
        
        total_sentences = 0
        curr_word_idx = 0
        r = 0
        
        while r < rows:
            # 1. Cycle Detection Logic
            if curr_word_idx in seen:
                prev_r, prev_count = seen[curr_word_idx]
                
                # Calculate the cycle properties
                cycle_len = r - prev_r
                sentences_in_cycle = total_sentences - prev_count
                
                # Calculate how many cycles we can jump
                num_cycles = (rows - r) // cycle_len
                
                total_sentences += num_cycles * sentences_in_cycle
                r += num_cycles * cycle_len
                
                # After jumping, clear 'seen' to prevent re-triggering the cycle
                seen = {} 
                if r >= rows: break

            # Record state before processing the row
            seen[curr_word_idx] = (r, total_sentences)

            # 2. Row Processing (with Memoization)
            if curr_word_idx not in memo:
                row_count = 0
                curr_col = 0
                temp_idx = curr_word_idx
                
                while curr_col + len(sentence[temp_idx]) <= cols:
                    curr_col += len(sentence[temp_idx]) + 1
                    temp_idx += 1
                    if temp_idx == n:
                        row_count += 1
                        temp_idx = 0
                
                # Edge Case: If no word could fit, return 0
                if temp_idx == curr_word_idx and row_count == 0:
                    return 0
                    
                memo[curr_word_idx] = (row_count, temp_idx)
            
            # Apply row results
            count, next_idx = memo[curr_word_idx]
            total_sentences += count
            curr_word_idx = next_idx
            r += 1
            
        return total_sentences

### Solution 3, String Transformation (The "Continuous Pointer" or "Formatted String" Method)
A clever way to turn this 2D fitting problem into a 1D string traversal problem.

-   **Core Idea:** Concatenate the entire sentence with spaces into a single string (e.g., `"hello world "`).
-   **Mechanism:** Maintain a pointer representing the total characters used on the screen. For each row, advance the pointer by `cols`.
-   **Adjustment:**
    -   If the pointer lands on a space, the words fit perfectly; move to the next character.
    -   If it lands in the middle of a word, you must move the pointer back to the start of that word because words cannot be split.
-   **Result:** The total number of times the sentence fits is `total_pointer_position / length_of_formatted_sentence`.
-   

The mathematical insight lies in **Periodic Mapping** and the relationship between **Quotient** and **Remainder** in integer division.

Imagine the master string s (length L) repeats infinitely. Any position P in this infinite sequence can be mapped back to the original string using the formula:  
P\=(q×L)+r

1.  **The Remainder (r\=P(modL)):**  
    This tells you the **current state**. Even as your `start` pointer grows into the thousands, the modulo operation maps it back to an index between 0 and L−1. This allows you to check which character of the sentence is "landing" at the end of a row without storing an infinite string.
    
2.  **The Quotient (q\=P//L):**  
    This tells you the **completed cycles**. Since each "cycle" of the string contains exactly one full sentence, dividing the total distance moved (`start`) by the length of one cycle (L) tells you exactly how many times the sentence has been completed.

### Complexity Analysis
- Time Complexity: O(L+rows * max_word_length)
- where L is the length of the concatenated string. In each row, you either move back a few characters or move forward 1.
  - String Construction (O(L)): Creating the master string s = " ".join(sentence) + " " involves iterating through every character of every word in the sentence once.
  - Main Loop (O(rows)): The code iterates through each row of the screen exactly once.
  - Pointer Adjustment (O(max_word_length)):
    - Inside the row loop, the while loop moves the pointer backward if it lands in the middle of a word. In the worst case, it moves back nearly the entire length of the word that was "cut off." Since a word cannot exceed cols, this adjustment is at most the length of the longest word in the sentence.

- Space Complexity: O(L) to store the concatenated string.

In [ ]:
class Solution:
    def wordsTyping(self, sentence: List[str], rows: int, cols: int) -> int:
        # 1. Create a single formatted string with spaces
        # By adding a space at the end of the sentence, you ensure that every word in the cycle is followed by exactly one space. 
        # This makes the period L consistent. Without it, the "gap" between the last word of one sentence and the first word of the next would be inconsistent, 
        # breaking the modulo logic.
        s = " ".join(sentence) + " "
        n = len(s)
        start = 0
        
        for _ in range(rows):
            # 2. Advance the pointer by the number of columns
            start += cols
            
            # 3. Check where we landed in the infinite string
            # This tells us exactly which character we are looking at after consuming start total spaces on the screen.
            if s[start % n] == ' ':
                # If we land on a space, the row ends perfectly; 
                # jump to the start of the next word.
                start += 1
            else:
                # If we land in the middle of a word, move the pointer 
                # backward to the beginning of that word.
                while start > 0 and s[(start - 1) % n] != ' ':
                    start -= 1
                    
        # 4. Total sentences = total characters consumed / length of one sentence
        return start // n